In [20]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [21]:
load_dotenv(override=True)
openai = OpenAI()

In [22]:
# For pushover 

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
  print(f"Pushover user found and starts with {pushover_user[0]}")
else:
  print("Pushover user not found")

if pushover_token:
  print(f"Pushover token found and starts with {pushover_token[0]}")
else:
  print("Pushover token not found")

Pushover user found and starts with u
Pushover token found and starts with a


In [23]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [24]:
push("hey")

Push: hey


In [25]:
# Used for when user wants to give their contact information
def record_user_details(email, name="Name not provided", notes="Not provided"):
  push(f"Recording interest from {name} with email and notes {notes} :3")
  return {"recorded": "ok"}

In [26]:
# Used to debug and help with adding information for when Flynn can't answer the question
def record_unknown_question(question):
  push(f"Recording: {question} | Wasn't able to answer question ")
  return {"recorded": "ok"}

In [27]:
record_user_details_json = {
  "name": "record_user_details",
  "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
  "parameters": {
    "type": "object",
    "properties": {
      "email": {
        "type": "string",
        "description": "The email address of this user"
      },
      "name": {
        "type": "string",
        "description": "The user's name, if they provided it"
      },
      "notes": {
        "type": "string",
        "description": "Any additional information about the conversation that's worth recording"
      }
    },
    "required": ["email"],
    "additionalProperties": False
  }
}

In [28]:
record_unknown_question_json = {
  "name": "record_unknown_question",
  "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
  "parameters": {
    "type": "object",
    "properties": {
      "question": {
        "type": "string",
        "description": "The question that couldn't be answered"
      },
    },
    "required": {"question"},
    "additionalProperties": False
  }
}

In [29]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [30]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being in touch and provided an email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address of this user'},
     'name': {'type': 'string',
      'description': "The user's name, if they provided it"},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': {'question'},
    'addi

In [31]:
def handle_tool_calls(tool_calls):
  results = []
  for tool_call in tool_calls:
      tool_name = tool_call.function.name
      arguments = json.loads(tool_call.function.arguments)
      print(f"Tool called: {tool_name}", flush=True)

      # THE BIG IF STATEMENT!!!

      if tool_name == "record_user_details":
          result = record_user_details(**arguments)
      elif tool_name == "record_unknown_question":
          result = record_unknown_question(**arguments)

      results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
  return results

In [32]:
globals()["record_unknown_question"]("this is a really hard question")

Push: Recording: this is a really hard question | Wasn't able to answer question 


{'recorded': 'ok'}

In [33]:
def handle_tool_calls(tool_calls):
  results = []
  for tool_call in tool_calls:
      tool_name = tool_call.function.name
      arguments = json.loads(tool_call.function.arguments)
      print(f"Tool called: {tool_name}", flush=True)
      tool = globals().get(tool_name)
      result = tool(**arguments) if tool else {}
      results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
  return results

In [34]:
reader = PdfReader("assets/linkedin.pdf")
linkedin = ""
for page in reader.pages:
  text = page.extract_text()
  if text:
    linkedin += text 

with open("assets/summary.txt", "r", encoding="utf-8") as f:
  summary = f.read()

with open("assets/projects.txt", "r", encoding="utf-8") as f:
  projects = f.read()

name = "Gabriel"
ai_Name = "Flynn"

In [35]:
system_prompt = f"You are acting as {ai_Name}, You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
You are responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile as well as projects from Github which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know how to answer to any question, use your record_unknown_question tool to record the questions that you couldn't answer. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details"

system_prompt += f"\n\n##Summary: \n {summary}\n\n##LinkedIn Profile:\n{linkedin}\n\n##Projects:\n{projects}"
system_prompt += f"With this contact, please chat with the suer, always staying in character."

In [36]:
system_prompt

"You are acting as Flynn, You are answering questions on Gabriel's website, particularly questions related to Gabriel's career, background, skills and experience. You are responsibility is to represent Gabriel for interactions on the website as faithfully as possible. You are given a summary of Gabriel's background and LinkedIn profile as well as projects from Github which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know how to answer to any question, use your record_unknown_question tool to record the questions that you couldn't answer. If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details\n\n##Summary: \n Hi! My name’s Gabriel Bello, and I’m currently studying Computer Systems at Simon Fraser University. I first got into Front-End Development through my CMPT 225 proje

In [ ]:
def chat(message, history):
  messages = [{"role": "system", "context": system_prompt}] + history + [{"role": "user", "context": message}]
  done = False
  while not done:
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

    finish_reason = response.choices[0].finish_reason

    if finish_reason=="tool_calls":
      message = response.choices[0].message
      tool_calls = message
      results = handle_tool_calls(message.tool_calls)
      messages.append(message)
      messages.extend(results)
    else:
      done = True
  return response.choices[0].message.content

In [38]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\johng\Documents\Flynn\.venv\Lib\site-packages\gradio\queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\johng\Documents\Flynn\.venv\Lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\johng\Documents\Flynn\.venv\Lib\site-packages\gradio\blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\johng\Documents\Flynn\.venv\Lib\site-packages\gradio\blocks.py", line 1621, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\johng\Documents\Flynn\.venv\Lib\site-packages\gradio\utils.py", line 882, in async_wrapper
    response = await f(*args, **kwargs)
    